# Prophet - Default approach

**Import Libraries and datasets**

In [1]:
!pip install prophet

     |████████████████████████████████| 65 kB 2.5 MB/s 
     |████████████████████████████████| 49 kB 3.9 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
  Created wheel for prophet: filename=prophet-1.0.1-py3-none-any.whl size=6638814 sha256=29efdb0477254f931ffb59ae5cf8df2013394a8886256e7f3ae243e47db138df
  Stored in directory: /root/.cache/pip/wheels/4e/a0/1a/02c9ec9e3e9de6bdbb3d769d11992a6926889d71567d6b9b67
Successfully built prophet
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 0.9.5
    Uninstalling cmdstanpy-0.9.5:
      Successfully uninstalled cmdstanpy-0.9.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fbprophet 0.7.1 requires cmdstanpy==0.9.5, but you have cmdstanpy 0.9.68 which is incompatible.


In [2]:
#Import Libraries
import pandas as pd
import math
import itertools
import numpy as np
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,mean_absolute_percentage_error
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric
from prophet.plot import plot_plotly, plot_components_plotly

In [ ]:
pd.set_option("display.max_columns",20)

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Github/m_tech/Datasets/input.xlsx')
df.head()

,Date,SalesVolume
0,1995-01-01,47639
1,1995-02-01,47880
2,1995-03-01,67025
3,1995-04-01,56925
4,1995-05-01,64192


In [ ]:
df = df.dropna()

In [3]:
df = pd.read_excel('/content/drive/MyDrive/Github/m_tech/Datasets/tfl-journeys-type.xlsx')
df = df.dropna()

In [4]:
df.tail()

,Date,Values
135,2021-07-01,98.5
136,2021-08-01,115.2
137,2021-09-01,129.8
138,2021-10-01,125.0
139,2021-11-01,131.0


**Model training and Prediction**

In [5]:
index = len(df) - 11
# input_df = df[:index].rename(columns={'Date':'ds', 'SalesVolume':'y'})
input_df = df[:index].rename(columns={'Date':'ds', 'Values':'y'})
input_df.tail(10)

,ds,y
119,2020-03-01,126.4
120,2020-04-01,31.3
121,2020-05-01,47.0
122,2020-06-01,66.7
123,2020-07-01,75.2
124,2020-08-01,94.8
125,2020-09-01,102.0
126,2020-10-01,88.9
127,2020-11-01,89.0
128,2020-12-01,57.5


In [6]:
#Train and fit the model
def model_train_predict(input_df,params):
  m = Prophet( seasonality_mode='additive', 
            yearly_seasonality= True,
            daily_seasonality = False,
            weekly_seasonality = False,
            **params)
  m.fit(input_df)
  # Forcasting into the future
  future = m.make_future_dataframe(periods=11,freq='MS')
  forecast = m.predict(future)
  return m, forecast

In [ ]:
default_params = {  
    'changepoint_prior_scale': 0.05,
    'changepoint_range' : 0.8,
    'seasonality_prior_scale': 10.0,
}

In [ ]:
model , forecast = model_train_predict(input_df,default_params)

In [ ]:
#Parameters for the model
model.component_modes

{'additive': ['yearly',
  'additive_terms',
  'extra_regressors_additive',
  'holidays'],
 'multiplicative': ['multiplicative_terms', 'extra_regressors_multiplicative']}

In [ ]:
# Plot the predictions
def viz_plot(m,forecast):
  fig = plot_plotly(m, forecast)
  fig.show()
  fig2 = plot_components_plotly(m, forecast)
  fig2.show()

In [ ]:
viz_plot(model , forecast)

# Cross Validation

In [ ]:
cutoffs = pd.date_range(start='2000-01-01', end='2020-06-01', freq='2MS')
cutoffs

In [ ]:
df_cv = cross_validation(m, horizon = '90 days', cutoffs = cutoffs, parallel="processes")

INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f589c2fdb90>


In [ ]:
df_p = performance_metrics(df_cv)
df_p.head()

In [ ]:
df_p

In [ ]:
fig = plot_cross_validation_metric(df_cv, metric='mape')


### **Evaluation Metrics**

In [7]:
def evaluation_metrics(forecast):
  metric_df = forecast[['ds','yhat']][311:318]
  metric_df['Actual'] = df['Values'][311:318]
  mae = mean_absolute_error(metric_df['yhat'], metric_df['Actual'])
  mape = mean_absolute_percentage_error(metric_df['yhat'], metric_df['Actual'])
  rmse = math.sqrt(mean_squared_error(metric_df['yhat'], metric_df['Actual']))
  print("Mean absolute error:", mae)
  print("Mean absolute percentage error:", mape)
  print("Root mean squared error:", rmse)

In [ ]:
evaluation_metrics(forecast)

ValueError: ignored

### **Hyper-Paramter Tuning**

**Tunable Parameters:**

***changepoint_prior_scale (Trend):*** 

This is probably the most impactful parameter. It determines the flexibility of the trend, and in particular how much the trend changes at the trend changepoints. As described in this documentation, if it is too small, the trend will be underfit and variance that should have been modeled with trend changes will instead end up being handled with the noise term. If it is too large, the trend will overfit and in the most extreme case you can end up with the trend capturing yearly seasonality. The default of 0.05 works for many time series, but this could be tuned; a range of [0.001, 0.5] would likely be about right. Parameters like this (regularization penalties; this is effectively a lasso penalty) are often tuned on a log scale.

***changepoint_range (Trend):*** 

This is the proportion of the history in which the trend is allowed to change. This defaults to 0.8, 80% of the history, meaning the model will not fit any trend changes in the last 20% of the time series. This is fairly conservative, to avoid overfitting to trend changes at the very end of the time series where there isn’t enough runway left to fit it well. With a human in the loop, this is something that can be identified pretty easily visually: one can pretty clearly see if the forecast is doing a bad job in the last 20%. In a fully-automated setting, it may be beneficial to be less conservative. It likely will not be possible to tune this parameter effectively with cross validation over cutoffs as described above. The ability of the model to generalize from a trend change in the last 10% of the time series will be hard to learn from looking at earlier cutoffs that may not have trend changes in the last 10%. So, this parameter is probably better not tuned, except perhaps over a large number of time series. In that setting, [0.8, 0.95] may be a reasonable range.

***seasonality_prior_scale:*** 

This parameter controls the flexibility of the seasonality. Similarly, a large value allows the seasonality to fit large fluctuations, a small value shrinks the magnitude of the seasonality. The default is 10., which applies basically no regularization. That is because we very rarely see overfitting here (there’s inherent regularization with the fact that it is being modeled with a truncated Fourier series, so it’s essentially low-pass filtered). A reasonable range for tuning it would probably be [0.01, 10]; when set to 0.01 you should find that the magnitude of seasonality is forced to be very small. This likely also makes sense on a log scale, since it is effectively an L2 penalty like in ridge regression.

***holidays_prior_scale:***

This controls flexibility to fit holiday effects. Similar to seasonality_prior_scale, it defaults to 10.0 which applies basically no regularization, since we usually have multiple observations of holidays and can do a good job of estimating their effects. This could also be tuned on a range of [0.01, 10] as with seasonality_prior_scale.

In [ ]:
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.4],
    'changepoint_range' : [0.8, 0.82, 0.85,0.9,0.92],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 5.0, 9.0],
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
maes = []  # Store the mae for each params here
mapes = []  # Store the mape for each params here
rmses = []  # Store the RMSEs for each params here

# Use cross validation to evaluate all parameters
for params in all_params:
    print(params)
    m = Prophet( daily_seasonality = False,
                weekly_seasonality = False, 
                **params).fit(input_df)  # Fit model with given params
    future = m.make_future_dataframe(periods=11,freq='MS')
    forecast = m.predict(future)
    # metric_df = forecast[['ds','yhat']][311:318]
    # metric_df['Actual'] = df['SalesVolume'][311:318]
    metric_df = forecast[['ds','yhat']][index:]
    metric_df['Actual'] = df['Values'][index:]
    mae = mean_absolute_error(metric_df['yhat'], metric_df['Actual'])
    mape = mean_absolute_percentage_error(metric_df['yhat'], metric_df['Actual'])
    rmse = math.sqrt(mean_squared_error(metric_df['yhat'], metric_df['Actual']))
    maes.append(mae)
    mapes.append(mape)
    rmses.append(rmse)

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['mae'] = maes
tuning_results['mape'] = mapes
tuning_results['rmse'] = rmses

{'changepoint_prior_scale': 0.001, 'changepoint_range': 0.8, 'seasonality_prior_scale': 0.01}


{'changepoint_prior_scale': 0.001, 'changepoint_range': 0.8, 'seasonality_prior_scale': 0.1}


{'changepoint_prior_scale': 0.001, 'changepoint_range': 0.8, 'seasonality_prior_scale': 1.0}


{'changepoint_prior_scale': 0.001, 'changepoint_range': 0.8, 'seasonality_prior_scale': 5.0}


{'changepoint_prior_scale': 0.001, 'changepoint_range': 0.8, 'seasonality_prior_scale': 9.0}


...

{'changepoint_prior_scale': 0.01, 'changepoint_range': 0.82, 'seasonality_prior_scale': 0.01}


{'changepoint_prior_scale': 0.01, 'changepoint_range': 0.82, 'seasonality_prior_scale': 0.1}


{'changepoint_prior_scale': 0.01, 'changepoint_range': 0.82, 'seasonality_prior_scale': 1.0}


{'changepoint_prior_scale': 0.01, 'changepoint_range': 0.82, 'seasonality_prior_scale': 5.0}


{'changepoint_prior_scale': 0.01, 'changepoint_range': 0.82, 'seasonality_prior_scale': 9.0}
{'changepoint_prior_scale': 0.01, 'changepoint_range': 0.85, 'seasonality_prior_scale': 0.01}
{'changepoint_prior_scale': 0.01, 'changepoint_range': 0.85, 'seasonality_prior_scale': 0.1}
{'changepoint_prior_scale': 0.01, 'changepoint_range': 0.85, 'seasonality_prior_scale': 1.0}
{'changepoint_prior_scale': 0.01, 'changepoint_range': 0.85, 'seasonality_prior_scale': 5.0}
{'changepoint_prior_scale': 0.01, 'changepoint_range': 0.85, 'seasonality_prior_scale': 9.0}
{'changepoint_prior_scale': 0.01, 'changepoint_range': 0.9, 'seasonality_prior_scale': 0.01}
{'changepoint_prior_scale': 0.01, 'changepoint_range': 0.9, 'seasonality_prior_scale': 0.1}
{'changepoint_prior_scale': 0.01, 'changepoint_range': 0.9, 'seasonality_prior_scale': 1.0}
{'changepoint_prior_scale': 0.01, 'changepoint_range': 0.9, 'seasonality_prior_scale': 5.0}
{'changepoint_prior_scale': 0.01, 'changepoint_range': 0.9, 'seasonality

In [ ]:
best_params = all_params[np.argmin(maes)]
print(best_params)

{'changepoint_prior_scale': 0.01, 'changepoint_range': 0.85, 'seasonality_prior_scale': 0.01}


In [ ]:
tuning_results.sort_values('mape').head()

,changepoint_prior_scale,changepoint_range,seasonality_prior_scale,mae,mape,rmse
35,0.01,0.85,0.01,33.902354,0.250616,44.295106
47,0.01,0.92,1.00,35.161840,0.254525,46.174368
49,0.01,0.92,9.00,38.860923,0.275040,48.865056
48,0.01,0.92,5.00,39.105308,0.276339,49.058560
44,0.01,0.90,9.00,39.776040,0.279860,49.680305


In [ ]:
tuning_results.sort_values('mae').head()

,changepoint_prior_scale,changepoint_range,seasonality_prior_scale,mae,mape,rmse
35,0.01,0.85,0.01,33.902354,0.250616,44.295106
47,0.01,0.92,1.00,35.161840,0.254525,46.174368
51,0.10,0.80,0.10,37.533536,0.469881,39.258788
50,0.10,0.80,0.01,38.535186,0.509679,40.889189
49,0.01,0.92,9.00,38.860923,0.275040,48.865056


### **Predicting with the best params**

In [ ]:
model , forecast = model_train_predict(input_df,best_params)

In [ ]:
viz_plot(model , forecast)

In [ ]:
evaluation_metrics(forecast)

Mean absolute error: 17195.636891038655
Mean absolute percentage error: 0.26045223006101204
Root mean squared error: 20930.967009333934
